# Sixth Stage

### Investigate into the impact of flash orders on the simulation results

In [1]:
from optimization import *
from flash_order import *

In [7]:
target_stock_code = '0050'
side = 'bid'
ts = 20
tm = 10
max_dur = 200
trick_thres = 100

In [8]:
# First get simulation results
sim_res = read_pkl_helper(os.path.join(PROJECT_DIR, f'optres/stock_code={target_stock_code}/side={side}/ts={ts}/tm={tm}', 'full_res.pkl'))

In [9]:
# Get flash orders
foc = FlashOrderCalculator(target_stock_code)
foc.classify(max_dur_ms=max_dur, trick_trade_thres_ms=trick_thres)

2022-12-06 16:34:20.354 | INFO     | get_data.py         :   36 | Got cache at /Users/anthonywah/Projects/mafs6100-order-placement-strategy/cache/0050.pkl - 8.80s
ClassifyingFlashOrder: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [00:12<00:00, 14.33it/s]


In [10]:
# For each simulation round, check if there is a flash order during the order placement period

for col in ['fo', 'fo_bid', 'fo_ask']:  # Record matched fo in index lists
    if col in sim_res.columns:
        sim_res = sim_res.drop(col, axis=1).reset_index(drop=True)
    sim_res.loc[:, col] = [[] for i in range(len(sim_res))]

one_fod = foc.fod[max_dur][trick_thres]
for fo in tqdm.tqdm(one_fod, desc="FindingFlashOrders", ncols=200, total=len(one_fod)):
    matched = sim_res.loc[(sim_res['start_ts'] <= fo['start_ms']) & (sim_res['fill_ts'] >= fo['end_ms'])].index
    if not len(matched):
        continue
    
    # Store case indexes
    for i in matched:
        sim_res.at[i, 'fo'].append(fo['case_index'])
    
    # Side breakdown
    if fo['side'] == 'bid':
        sim_res.at[i, 'fo_bid'].append(fo['case_index'])
    else: 
        sim_res.at[i, 'fo_ask'].append(fo['case_index'])

FindingFlashOrders: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10434/10434 [00:07<00:00, 1382.55it/s]


In [11]:
# Get simulation rounds with flash orders
sim_res_dict = {
    'Normal'.ljust(15): sim_res.copy(), 
    'With FO'.ljust(15): sim_res.loc[sim_res['fo'].apply(len) > 0, :].reset_index(drop=True), 
    'No FO'.ljust(15): sim_res.loc[sim_res['fo'].apply(len) > 0, :].reset_index(drop=True), 
    'With bid FO'.ljust(15): sim_res.loc[sim_res['fo_bid'].apply(len) > 0, :].reset_index(drop=True), 
    'No bid FO'.ljust(15): sim_res.loc[sim_res['fo_bid'].apply(len) == 0, :].reset_index(drop=True), 
    'With ask FO'.ljust(15): sim_res.loc[sim_res['fo_ask'].apply(len) > 0, :].reset_index(drop=True), 
    'No ask FO'.ljust(15): sim_res.loc[sim_res['fo_ask'].apply(len) == 0, :].reset_index(drop=True), 
}

In [14]:
# Calculation % of different fill types
log_info(f'Cases breakdown: ')
for k, i_sim_res in sim_res_dict.items():
    count_dict = i_sim_res.groupby('case')['date'].count().to_dict()
    msg = f'{k}: Count = {len(i_sim_res):>7}'
    for case in ['INIT', 'EOQREP', 'REP', 'TAKE']:
        msg += f' ; {case} = {count_dict.get(case, 0.) * 100 / len(sim_res):>6.2f}%'
    msg += f' ; Avg PnL = {i_sim_res["pnl"].mean():>6.3f} ; Stdev = {i_sim_res["pnl"].std():>6.3f}'
    log_info(msg)

2022-12-06 16:35:34.914 | INFO     | 3074678047.py       :    2 | Cases breakdown: 
2022-12-06 16:35:34.925 | INFO     | 3074678047.py       :    9 | Normal         : Count =   49945 ; INIT =   5.75% ; EOQREP =   4.69% ; REP =  28.66% ; TAKE =  60.90% ; Avg PnL = -2.085 ; Stdev =  3.115
2022-12-06 16:35:34.929 | INFO     | 3074678047.py       :    9 | With FO        : Count =    4321 ; INIT =   0.69% ; EOQREP =   0.42% ; REP =   2.96% ; TAKE =   4.59% ; Avg PnL = -2.456 ; Stdev =  4.471
2022-12-06 16:35:34.932 | INFO     | 3074678047.py       :    9 | No FO          : Count =    4321 ; INIT =   0.69% ; EOQREP =   0.42% ; REP =   2.96% ; TAKE =   4.59% ; Avg PnL = -2.456 ; Stdev =  4.471
2022-12-06 16:35:34.934 | INFO     | 3074678047.py       :    9 | With bid FO    : Count =    1450 ; INIT =   0.13% ; EOQREP =   0.15% ; REP =   0.59% ; TAKE =   2.04% ; Avg PnL = -4.666 ; Stdev =  5.257
2022-12-06 16:35:34.946 | INFO     | 3074678047.py       :    9 | No bid FO      : Count =   48495 ;

In [13]:
one_fod[0]

{'date': '2022-01-03',
 'side': 'bid',
 'start': Timestamp('2022-01-03 09:00:12.134000'),
 'start_ms': 1641200412134.0,
 'start_index': 35,
 'end': Timestamp('2022-01-03 09:00:12.174000'),
 'end_ms': 1641200412174.0,
 'end_index': 40,
 'duration': 40.0,
 'orig_price': 14615.0,
 'fo_price': 14620.0,
 'fo_qty': 17,
 'tricked_trade': False,
 'tricked': nan,
 'tricked_index': nan,
 'tricked_ms': nan,
 'tricked_price': nan,
 'tricked_qty': nan,
 'tricked_ms_taken': nan,
 'opp_tricked_trade': False,
 'opp_tricked': nan,
 'opp_tricked_index': nan,
 'opp_tricked_ms': nan,
 'opp_tricked_price': nan,
 'opp_tricked_qty': nan,
 'opp_tricked_ms_taken': nan,
 'case_index': 0}